# Model

- XGBoost (use Chatgpt for the framework of classification)

# Features

- Pct Change of Meat Production (1m, 3m, 6m)
- stock autocorrelation
- cold storage
- determinlistic process (add later to see if any improvement)

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [9]:
stocks = pd.read_csv("Data/all_stock_and_etfs.csv", index_col=0)
MCD = stocks[stocks.Ticker_Symbol == "MCD"]

In [10]:
display(MCD)

,Open,High,Low,Close,Volume,Ticker_Symbol
Date-Time,,,,,,
2024-02-20,292.00,295.4999,291.84,292.65,2263994,MCD
2024-02-16,290.79,293.7000,289.26,292.02,2837573,MCD
2024-02-15,287.90,292.1000,287.28,291.86,2401617,MCD
2024-02-14,287.32,288.3200,286.59,288.17,2139326,MCD
2024-02-13,288.63,289.9000,285.67,287.11,2756190,MCD
...,...,...,...,...,...,...
1999-11-05,45.69,46.6300,45.63,46.50,7184900,MCD
1999-11-04,43.69,45.2500,43.44,45.19,9834800,MCD
1999-11-03,41.25,43.4400,41.06,41.94,7600400,MCD


In [11]:
MCD = MCD[::-1]

In [12]:
MCD['Shifted'] = MCD['Close'].shift(-30)
MCD['Signal'] = (MCD['Close'] < MCD['Shifted']).astype(int)
MCD = MCD.drop('Shifted', axis=1)
display(MCD)

,Open,High,Low,Close,Volume,Ticker_Symbol,Signal
Date-Time,,,,,,,
1999-11-01,41.75,42.0000,41.25,41.69,3932900,MCD,0
1999-11-02,42.06,42.1300,41.06,41.25,2250000,MCD,1
1999-11-03,41.25,43.4400,41.06,41.94,7600400,MCD,0
1999-11-04,43.69,45.2500,43.44,45.19,9834800,MCD,0
1999-11-05,45.69,46.6300,45.63,46.50,7184900,MCD,0
...,...,...,...,...,...,...,...
2024-02-13,288.63,289.9000,285.67,287.11,2756190,MCD,0
2024-02-14,287.32,288.3200,286.59,288.17,2139326,MCD,0
2024-02-15,287.90,292.1000,287.28,291.86,2401617,MCD,0


In [13]:
MCD['1m_pct_change'] = MCD['Close'].pct_change(periods=30)
MCD['3m_pct_change'] = MCD['Close'].pct_change(periods=90)
MCD['6m_pct_change'] = MCD['Close'].pct_change(periods=180)

In [15]:
MCD['Return'] = MCD['Close'].pct_change()
MCD['1m_volatility'] = MCD['Return'].rolling(window=30).std() * np.sqrt(30)
MCD['3m_volatility'] = MCD['Return'].rolling(window=90).std() * np.sqrt(90)
MCD['6m_volatility'] = MCD['Return'].rolling(window=180).std() * np.sqrt(180)

In [16]:
display(MCD)

,Open,High,Low,Close,Volume,Ticker_Symbol,Signal,1m_pct_change,3m_pct_change,6m_pct_change,Return,1m_volatility,3m_volatility,6m_volatility
Date-Time,,,,,,,,,,,,,,
1999-11-01,41.75,42.0000,41.25,41.69,3932900,MCD,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-02,42.06,42.1300,41.06,41.25,2250000,MCD,1,NaN,NaN,NaN,-0.010554,NaN,NaN,NaN
1999-11-03,41.25,43.4400,41.06,41.94,7600400,MCD,0,NaN,NaN,NaN,0.016727,NaN,NaN,NaN
1999-11-04,43.69,45.2500,43.44,45.19,9834800,MCD,0,NaN,NaN,NaN,0.077492,NaN,NaN,NaN
1999-11-05,45.69,46.6300,45.63,46.50,7184900,MCD,0,NaN,NaN,NaN,0.028989,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-13,288.63,289.9000,285.67,287.11,2756190,MCD,0,-0.031702,0.122356,0.005569,-0.008050,0.063617,0.093882,0.122408
2024-02-14,287.32,288.3200,286.59,288.17,2139326,MCD,0,-0.029861,0.142489,0.007447,0.003692,0.063732,0.092619,0.122449
2024-02-15,287.90,292.1000,287.28,291.86,2401617,MCD,0,-0.008594,0.175812,0.024358,0.012805,0.064616,0.091600,0.123040


In [18]:
data = MCD
data = data.drop(['Open', 'High', 'Low', 'Volume', 'Ticker_Symbol', 'Close'], axis=1)